In [1]:
import findspark
findspark.init()
import os
import pyspark
import sys
from pyspark.sql import SparkSession

In [2]:
## Start Spark Session
spark = SparkSession.builder.appName('app_name') \
        .config('spark.jars.packages','org.apache.hudi:hudi-spark3.3-bundle_2.12:0.13.1') \
        .config('spark.serializer','org.apache.spark.serializer.KryoSerializer') \
        .config('spark.sql.extensions','org.apache.spark.sql.hudi.HoodieSparkSessionExtension') \
        .config('spark.sql.catalog.spark_catalog','org.apache.spark.sql.hudi.catalog.HoodieCatalog') \
        .getOrCreate()
print("Spark Running")

:: loading settings :: url = jar:file:/opt/anaconda3/envs/spark_hudi/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/anshumanr/.ivy2/cache
The jars for the packages stored in: /Users/anshumanr/.ivy2/jars
org.apache.hudi#hudi-spark3.3-bundle_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3730f5f0-32d1-4fae-95ca-d75af4cc64da;1.0
	confs: [default]
	found org.apache.hudi#hudi-spark3.3-bundle_2.12;0.13.1 in central
:: resolution report :: resolve 247ms :: artifacts dl 11ms
	:: modules in use:
	org.apache.hudi#hudi-spark3.3-bundle_2.12;0.13.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-sub

23/08/31 16:04:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/08/31 16:04:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
Spark Running


## Creating Table

In [3]:
tablename='Table1'
basePath = "/Users/anshumanr/Documents/Hudi/Apache_Hudi_Spark/My_warehouse/Table1"
dataGen = spark._jvm.org.apache.hudi.QuickstartUtils.DataGenerator()

In [4]:
inserts = spark._jvm.org.apache.hudi.QuickstartUtils.convertToStringList(dataGen.generateInserts(10))
df = spark.read.json(spark.sparkContext.parallelize(inserts, 2))

In [5]:
hudi_options = {
    'hoodie.table.name': tablename,
    'hoodie.datasource.write.recordkey.field': 'uuid',
    'hoodie.datasource.write.table.name': tablename,
    'hoodie.datasource.write.operation': 'upsert',
    'hoodie.datasource.write.precombine.field': 'ts'
}

In [6]:
df.write.format("hudi"). \
    options(**hudi_options). \
    mode("overwrite"). \
    save(basePath)

23/08/31 16:04:21 WARN DFSPropertiesConfiguration: Cannot find HUDI_CONF_DIR, please set it as the dir of hudi-defaults.conf
23/08/31 16:04:21 WARN DFSPropertiesConfiguration: Properties file file:/etc/hudi/conf/hudi-defaults.conf not found. Ignoring to load props file
23/08/31 16:04:22 WARN HoodieBackedTableMetadata: Metadata table was not found at path /Users/anshumanr/Documents/Hudi/Apache_Hudi_Spark/My_warehouse/Table1/.hoodie/metadata


# WARNING: Unable to get Instrumentation. Dynamic Attach failed. You may add this JAR as -javaagent manually, or supply -Djdk.attach.allowAttachSelf
# WARNING: Unable to attach Serviceability Agent. Unable to attach even with module exceptions: [org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed., org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed., org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed.]
23/08/31 16:04:39 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-hbase.properties,hadoop-metrics2.properties


## Querying Data

In [7]:
tripsSnapshotDF = spark. \
  read. \
  format("hudi"). \
  load(basePath)

In [8]:
tripsSnapshotDF.createOrReplaceTempView("hudi_trips_snapshot")

In [9]:
spark.sql("select fare, begin_lon, begin_lat, ts from  hudi_trips_snapshot where fare > 20.0").show()

+------------------+-------------------+-------------------+-------------+
|              fare|          begin_lon|          begin_lat|           ts|
+------------------+-------------------+-------------------+-------------+
|  43.4923811219014| 0.8779402295427752| 0.6100070562136587|1693211548369|
|34.158284716382845|0.46157858450465483| 0.4726905879569653|1693273667487|
| 66.62084366450246|0.03844104444445928| 0.0750588760043035|1693002314088|
| 41.06290929046368| 0.8192868687714224|  0.651058505660742|1693221471879|
| 93.56018115236618|0.14285051259466197|0.21624150367601136|1693443534075|
| 33.92216483948643| 0.9694586417848392| 0.1856488085068272|1693381867365|
| 27.79478688582596| 0.6273212202489661|0.11488393157088261|1692984205507|
| 64.27696295884016| 0.4923479652912024| 0.5731835407930634|1693255851431|
+------------------+-------------------+-------------------+-------------+



In [10]:
spark.sql("select _hoodie_commit_time, _hoodie_record_key, _hoodie_partition_path, rider, driver, fare from  hudi_trips_snapshot").show()

+-------------------+--------------------+----------------------+---------+----------+------------------+
|_hoodie_commit_time|  _hoodie_record_key|_hoodie_partition_path|    rider|    driver|              fare|
+-------------------+--------------------+----------------------+---------+----------+------------------+
|  20230831160421605|0247e363-2171-48a...|                      |rider-213|driver-213|  43.4923811219014|
|  20230831160421605|67fe72df-2f94-433...|                      |rider-213|driver-213|34.158284716382845|
|  20230831160421605|0d4253a0-ed77-4f3...|                      |rider-213|driver-213|17.851135255091155|
|  20230831160421605|b75a89e8-7957-4e1...|                      |rider-213|driver-213| 66.62084366450246|
|  20230831160421605|c91287b6-9617-4ba...|                      |rider-213|driver-213| 41.06290929046368|
|  20230831160421605|e6df7cf1-06af-492...|                      |rider-213|driver-213|19.179139106643607|
|  20230831160421605|142f467c-afcb-458...|    